### KEYS

In [ ]:
import json
import pandas as pd
import numpy as np
from numpy import random
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from matplotlib.gridspec import GridSpec
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import mannwhitneyu
from scipy.stats import fligner
from scipy.stats import skewtest
stats.junk = lambda chisq, df: stats.chi2.sf(chisq, df)
import csv
import gffpandas.gffpandas as gffpd
import sklearn
from sklearn.metrics import r2_score
import os
import statistics
%load_ext rpy2.ipython

#### DATAFRAME RECORDING THE GENERATION TIME PER SAMPLE ####
dic_gen = {'CC2344-L1': 912.356113, 'CC2344-L10': 917.129696, 'CC2344-L11': 889.5859554, 'CC2344-L12': 950.0552184, 'CC2344-L13': 961.4186064,
           'CC2344-L14': 931.447801, 'CC2344-L15': 946.6643063, 'CC2344-L2': 923.1078072, 'CC2344-L3': 1000.469526, 'CC2344-L4': 808.9505794,
           'CC2344-L5': 957.6380465, 'CC2344-L6': 970.6307256, 'CC2344-L7': 990.9451516, 'CC2344-L8': 1009.966123, 'CC2344-L9': 901.0619061, 
           'CC2931-L1': 1050.109001, 'CC2931-L10': 1097.978141, 'CC2931-L11': 1021.13559, 'CC2931-L13': 1041.362593, 
           'CC2931-L14': 1016.111493, 'CC2931-L15': 1052.540951, 'CC2931-L2': 1056.765369, 'CC2931-L3': 1000.399127, 'CC2931-L4': 1011.411706,
           'CC2931-L5': 993.8603657, 'CC2931-L6': 1083.095655, 'CC2931-L7': 1067.34507, 'CC2931-L9': 1079.236285}
generations = pd.Series(dic_gen)

#### DATAFRAME RECORDING THE NUMBER OF MUTATIONS PER SAMPLE ####
dic_mut = {'CC2344-L1': 396, 'CC2344-L10': 59, 'CC2344-L11': 46, 'CC2344-L12': 74, 'CC2344-L13': 49, 'CC2344-L14': 46, 'CC2344-L15': 53, 
           'CC2344-L2': 80, 'CC2344-L3': 63, 'CC2344-L4': 24, 'CC2344-L5': 68, 'CC2344-L6': 38, 'CC2344-L7': 45, 'CC2344-L8': 75, 'CC2344-L9': 27, 
           'CC2931-L1': 89, 'CC2931-L10': 87, 'CC2931-L11': 85, 'CC2931-L13': 97, 'CC2931-L14': 79, 'CC2931-L15': 141, 'CC2931-L2': 123,
           'CC2931-L3': 52, 'CC2931-L4': 100, 'CC2931-L5': 335, 'CC2931-L6': 84, 'CC2931-L7': 72, 'CC2931-L9': 113}
mutations = pd.Series(dic_mut)

### COMBINING UNNORMALIZED RAW HTSEQ COUNTS

In [ ]:
## NAMES OF GENE COUNT FILES ##
keyword = 'genes_count'
label = []
for fname in os.listdir('/research/projects/chlamydomonas/MAexpression/data/gene_count/'):
    if keyword in fname:
        fname = fname.replace('_genes_count','')
        if 'CC2344' in fname:
            label.append(fname)
            
# CC2344 - UNNORMALIZED GENE COUNTS ##
gene_count = pd.DataFrame()
for i in label:
    section = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/gene_count/' + i + '_genes_count', delimiter = '\t', names = ['name', i])
    gene_count = pd.concat([gene_count, section], axis = 1)
gene_count = gene_count.loc[:,~gene_count.columns.duplicated()]
gene_count = gene_count.drop(gene_count.index.values[-5:])
gene_count = gene_count.set_index('name')

## EXPORTING 
gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2344_raw.txt', sep = '\t', index = True, header = True)

##################################################################################################################################

## NAMES OF GENE COUNT FILES ##
keyword = 'genes_count'
label = []
for fname in os.listdir('/research/projects/chlamydomonas/MAexpression/data/gene_count/'):
    if keyword in fname:
        fname = fname.replace('_genes_count','')
        if 'CC2931' in fname:
            label.append(fname)

## CC2931 - UNNORMALIZED GENE COUNTS ##
gene_count = pd.DataFrame()
for i in label:
    section = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/gene_count/' + i + '_genes_count', delimiter = '\t', names = ['name', i])
    gene_count = pd.concat([gene_count, section], axis = 1)
    
gene_count = gene_count.loc[:,~gene_count.columns.duplicated()]
gene_count = gene_count.drop(gene_count.index.values[-5:])
gene_count = gene_count.set_index('name')

## EXPORTING
gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2931_raw.txt', sep = '\t', index = True, header = True)

### GATHERING FRAGMENT LENGTHS FROM GFF FILE

In [ ]:
gff = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v6_genome_plus_anno/CC4532.v1_1.gene_exons.gff3', skiprows=3, delimiter = '\t')
gff.columns = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
gff[['attributes1', 'attributes2', 'attributes3', 'attributes4']] = gff['attributes'].str.split(';', n = 3, expand = True)
gff_exon = gff.loc[gff['type'].isin(['mRNA', 'exon'])]
gff_exon[['attributes1', 'attributes2', 'attributes4']] = gff_exon[['attributes1', 'attributes2', 'attributes4']].replace({'Parent=':'', 'ID=':''}, regex = True)
gff_exon_pos = gff_exon.loc[gff_exon['strand'] == '+']
gff_exon_neg = gff_exon.loc[gff_exon['strand'] == '-']
mRNA = gff_exon.loc[gff_exon['type'] == 'mRNA'].reset_index()
mRNA = mRNA.drop('index', axis = 1)
mRNA['attributes'] = mRNA['attributes'].str.split(';')
mRNA['parent'] = 'nan'
for i in list(mRNA.index.values):
    for a in mRNA.at[i,'attributes']:
        if 'Parent=' in a:
            mRNA.at[i, 'parent'] = a

## EXTRACTING THE TOTAL SUM OF EXONS PER GENE
mRNA['total_exon_length'] = 'nan'
mRNA.rename(columns={'attributes1': 'ID'}, inplace = True)
mRNA_ID = mRNA['ID'].values.tolist()
gff_exon = gff_exon.loc[gff_exon['type'] == 'exon']

for i in range(len(mRNA_ID)):
    gene_length = []
    section = gff_exon.loc[gff_exon['attributes2'] == mRNA_ID[i]]
    for a in list(section.index.values):
        gene_length.append(section.at[a,'end'] - section.at[a, 'start'])
    mRNA.at[i, 'total_exon_length'] = sum(gene_length)
mRNA_1 = mRNA.drop_duplicates(subset=['parent'], keep='last')

fragment_length = pd.Series(data = mRNA_1['total_exon_length'].values.tolist(), index = list(parent), name = 'total_exon_length')

## EXPORT DATA
fragment_length.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', sep = '\t', index = True, index_label= 'index', header = True)

### INSTALLING PACKAGES AND LOADING LIBRARIES

In [ ]:
%%R
## TIDYVERSE
library(tidyverse)

## DESEQ2
library("BiocParallel")
register(MulticoreParam(4))
library("DESeq2")

## CQN
library(cqn)
library(scales)

## NOISEQ
library(NOISeq)

## NOISEQ

### PRE-FILTERING THROUGH NOISEQ

In [ ]:
%%R

#######################################
### NOISEQ PRE-FILTERING FOR CC2931 ###
#######################################
CC2931 <- data.frame(read.csv('/research//projects//chlamydomonas//MAexpression//analysis//raw_counts//unnormalized_CC2931_raw.txt', sep = '\t', header = TRUE, row.names = 'name', stringsAsFactors = FALSE))
gene_length <- data.frame(read.csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', sep = '\t', header = TRUE, stringsAsFactors = FALSE))
factor <- colnames(CC2931)[1:length(CC2931)]
factor[grep('ANC', factor)] <- 'control'
factor[grep('L', factor)] <- 'treatment'
factor <- matrix(factor, nrow = length(CC2931), ncol = 1)
filtered_CC2931 <- filtered.data(CC2931, factor, norm=FALSE, method=2, p.adj='BH')

#######################################
### NOISEQ PRE-FILTERING FOR CC2344 ###
#######################################
CC2344 <- data.frame(read.table('/research//projects//chlamydomonas//MAexpression//analysis//raw_counts//unnormalized_CC2344_raw.txt', sep = '\t', header = TRUE, row.names = 'name', stringsAsFactors = FALSE))
factor <- colnames(CC2344)[1:length(CC2344)]
factor[grep('ANC', factor)] <- 'control'
factor[grep('L', factor)] <- 'treatment'
factor <- matrix(factor, nrow = length(CC2344), ncol = 1)
filtered_CC2344 <- filtered.data(CC2344, factor, norm=FALSE, method=2, p.adj='BH')

## EXPORTING PREFILTERED TEXTFILE
write.table(as.data.frame(filtered_CC2931), 'raw_counts//noiseq_filtered_unnormalized_CC2931.txt', sep="\t", quote = F)
write.table(as.data.frame(filtered_CC2344), 'raw_counts//noiseq_filtered_unnormalized_CC2344.txt', sep="\t", quote = F)

### FILTERED GC CONTENT

In [ ]:
## OPENING GC LENGTH AND FRAGMENT LENGTH
gc_content = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v6_genome_plus_anno/GC_lengths.tsv', delimiter = '\t')
fragment_length = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', delimiter = '\t')
fragment_length = fragment_length.set_index('Unnamed: 0')

# OPENING NOISEQ FILTERED GENE COUNTS
CC2931_gene_count = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/noiseq_filtered_unnormalized_CC2931.txt', delimiter = '\t')
CC2344_gene_count = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/noiseq_filtered_unnormalized_CC2344.txt', delimiter = '\t')

CC2931_gene_count = CC2931_gene_count.join([fragment_length, gc_content['GC']])
CC2344_gene_count = CC2344_gene_count.join([fragment_length, gc_content['GC']])

# EXPORT ##
CC2344_gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_filtered_noiseq_wt_GC', sep = '\t', index = True, header = True)
CC2931_gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_filtered_noiseq_wt_GC', sep = '\t', index = True, header = True)

### DESEQ NORMALIZING NOISEQ FILTERED DATA

In [ ]:
%%R

################
#### CC2931 ####
################
CC2931 <- data.frame(read.csv('/research//projects//chlamydomonas//MAexpression//analysis//raw_counts//CC2931_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, row.names = 'X', stringsAsFactors = FALSE))
condition <- colnames(CC2931)[1:(length(CC2931)-2)]
counts <- CC2931[condition]
condition_1 <- condition
condition[grep('ANC', condition)] <- 'control'
condition[grep('L', condition)] <- 'treatment'

coldata <- data.frame(condition, condition_1, row.names = 'condition_1')

dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ condition)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
dds <- estimateSizeFactors(dds)
counts <- counts(dds, normalized=TRUE)
write.table(counts, paste(directory, "CC2931_noiseq_filtered_deseq_normalized", ".txt", sep=""), sep="\t", quote = F)

################
#### CC2344 ####
################
CC2344 <- data.frame(read.csv('/research//projects//chlamydomonas//MAexpression//analysis//raw_counts//CC2344_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, row.names = 'X', stringsAsFactors = FALSE))
condition <- colnames(CC2344)[1:(length(CC2344)-2)]
counts <- CC2344[condition]
condition_1 <- condition
condition[grep('ANC', condition)] <- 'control'
condition[grep('L', condition)] <- 'treatment'

coldata <- data.frame(condition, condition_1, row.names = 'condition_1')

dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ condition)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
dds <- estimateSizeFactors(dds)
counts <- counts(dds, normalized=TRUE)
write.table(counts, paste(directory, "CC2344_noiseq_filtered_deseq_normalized", ".txt", sep=""), sep="\t", quote = F)

### ADDING GC CONTENT TO DESEQ2 NORMALIZED NOISEQ FILTERED GENE COUNTS

In [ ]:
## OPENING DESEQ2 NORMALIZED NOISEQ FILTERED GENE COUNTS
CC2931_normalized_GC = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_noiseq_filtered_deseq_normalized.txt', delimiter = '\t')
CC2344_normalized_GC = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_noiseq_filtered_deseq_normalized.txt', delimiter = '\t')

CC2931_normalized_GC = CC2931_normalized_GC.join([fragment_length, gc_content['GC']])
CC2344_normalized_GC = CC2344_normalized_GC.join([fragment_length, gc_content['GC']])

## EXPORT ##
CC2931_normalized_GC.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_deseq_normalized_noiseq_filtered_wt_GC', sep = '\t', index = True, header = True)
CC2344_normalized_GC.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_deseq_normalized_noiseq_filtered_wt_GC', sep = '\t', index = True, header = True)

### CREATING NOISEQ DATA OBJECT

In [ ]:
%%R
#####################################
### CREATING A NOISEQ DATA OBJECT ###
#####################################
CC2931_deseq_normalized <- data.frame(read.table('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_deseq_normalized_noiseq_filtered_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE))
CC2931_length <- data.frame(CC2931_deseq_normalized$total_exon_length)
CC2931_GC <- data.frame(CC2931_deseq_normalized$GC)

## PROVIDING INDEXES
row.names(CC2931_length) <- CC2931_deseq_normalized$X
row.names(CC2931_GC) <- CC2931_deseq_normalized$X
row.names(CC2931_deseq_normalized) <- CC2931_deseq_normalized$X

## CREATING FACTORS AND COUNTS
CC2931_counts <- CC2931_deseq_normalized[colnames(CC2931_deseq_normalized)[2:(length(CC2931_deseq_normalized)-2)]]
CC2931_factor <- colnames(CC2931_counts)
CC2931_columns <- CC2931_factor
colnames(CC2931_counts) <- CC2931_columns
CC2931_factor[grep('ANC', CC2931_factor)] <- 'control'
CC2931_factor[grep('L', CC2931_factor)] <- 'treatment'
num <- c(1:length(t(CC2931_factor)))
CC2931_factor <- data.frame(num, CC2931_factor, CC2931_columns, row.names = 'num')
##############################################################################################################################

CC2344_deseq_normalized <- data.frame(read.table('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_deseq_normalized_noiseq_filtered_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE))
CC2344_length <- data.frame(CC2344_deseq_normalized$total_exon_length)
CC2344_GC <- data.frame(CC2344_deseq_normalized$GC)

## PROVIDING INDEXES
row.names(CC2344_length) <- CC2344_deseq_normalized$X
row.names(CC2344_GC) <- CC2344_deseq_normalized$X
row.names(CC2344_deseq_normalized) <- CC2344_deseq_normalized$X

## CREATING FACTORS AND COUNTS
CC2344_counts <- CC2344_deseq_normalized[colnames(CC2344_deseq_normalized)[2:(length(CC2344_deseq_normalized)-2)]]
CC2344_factor <- colnames(CC2344_counts)
CC2344_columns <- CC2344_factor
colnames(CC2344_counts) <- CC2344_columns
CC2344_factor[grep('ANC', CC2344_factor)] <- 'control'
CC2344_factor[grep('L', CC2344_factor)] <- 'treatment'
CC2344_factor <- data.frame(CC2344_factor)


## CREATING READDATA FILES
data_CC2931 <- readData(data = CC2931_counts, length = CC2931_length, gc = CC2931_GC, CC2931_factor)
data_CC2344 <- readData(data = CC2344_counts, length = CC2344_length, gc = CC2344_GC, factors = CC2344_factor)

## CONDITIONAL QUANTILE NORMALIZATION

In [ ]:
###################################
### CC2931 - CQN NORMALIZATION ####
###################################
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
CC2931_noiseq_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'X')
counts <- CC2931_noiseq_filtered[c(1:42)]
cqn_res <- cqn(counts = counts, x = CC2931_noiseq_filtered$GC, lengths = CC2931_noiseq_filtered$total_exon_length)

#### CREATING FACTORS AND COLDATA ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L9', 'L10', 'L11', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### CREATING PLOT ####
jpeg(file="cqn/CC2931_gc_plot_before.jpeg")
cqnplot(cqn_res, n = 1, xlab = 'GC content', ylim = c(5,10))
dev.off()

jpeg(file="cqn/CC2931_length_plot_before.jpeg")
cqnplot(cqn_res, n = 2, xlim = c(-2,4.5), xlab = 'length', ylim = c(5.5,8.5))
dev.off()

#### NORMALIZED EXPRESSION VALUES ####
cqnnorm <- cqn_res$y + cqn_res$offset
cqnOffset <- cqn_res$glm.offset
cqnNormFactors <- exp(cqnOffset)
cqnNormFactors[is.na(cqnNormFactors)] <- 1 ## replacing missing values with 1
normFactors_sameScale <- cqnNormFactors/exp(rowMeans(log(cqnNormFactors)))

#### CREATE DATASET USING NORMFACTORS_SAMESCALE ####
CC2931_dds1 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
normalizationFactors(CC2931_dds1) <- normFactors_sameScale
counts_nobias_sameScale = counts(CC2931_dds1, normalized = TRUE)
write.table(counts_nobias_sameScale, paste(directory, "CC2931_noiseq_filtered_cqn_normalized_samescale", ".txt", sep=""), sep="\t", quote = F)

#### CREATE DATASET USING NORMFACTORS ####
CC2931_dds2 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
normalizationFactors(CC2931_dds2) = cqnNormFactors
counts_nobias = counts(CC2931_dds2, normalized = TRUE)
write.table(counts_nobias, paste(directory, "CC2931_noiseq_filtered_cqn_normalized", ".txt", sep=""), sep="\t", quote = F)

In [ ]:
####################################
#### CC2344 - CQN NORMALIZATION ####
####################################
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
CC2344_noiseq_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'X')
counts <- CC2344_noiseq_filtered[c(1:48)]
cqn_res <- cqn(counts = counts, x = CC2344_noiseq_filtered$GC, lengths = CC2344_noiseq_filtered$total_exon_length)

#### CREATING FACTORS AND COLDATA ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### CREATING PLOT ####
jpeg(file = "cqn/CC2344_gc_plot_before.jpeg")
cqnplot(cqn_res, n = 1, xlab = 'GC content', ylim = c(3, 10))
dev.off()

jpeg(file = "cqn/CC2344_length_plot_before.jpeg")
cqnplot(cqn_res, n = 2, xlim = c(-2,4.5), xlab = 'length', ylim = c(4, 8))
dev.off()

#### NORMALIZED EXPRESSION VALUES ####
cqnnorm <- cqn_res$y + cqn_res$offset
cqnOffset <- cqn_res$glm.offset
cqnNormFactors <- exp(cqnOffset)
cqnNormFactors[is.na(cqnNormFactors)] <- 1 ## replacing missing values with 1
normFactors_sameScale <- cqnNormFactors/exp(rowMeans(log(cqnNormFactors)))

#### CREATE DATASET USING NORMFACTORS_SAMESCALE ####
CC2344_dds1 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
normalizationFactors(CC2344_dds1) <- normFactors_sameScale
counts_nobias_sameScale = counts(CC2344_dds1, normalized = TRUE)
write.table(counts_nobias_sameScale, paste(directory, "CC2344_noiseq_filtered_cqn_normalized_samescale", ".txt", sep=""), sep="\t", quote = F)

#### CREATE DATASET USING NORMFACTORS ####
CC2344_dds2 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
normalizationFactors(CC2344_dds2) = cqnNormFactors
counts_nobias = counts(CC2344_dds2, normalized = TRUE)
write.table(counts_nobias, paste(directory, "CC2344_noiseq_filtered_cqn_normalized", ".txt", sep=""), sep="\t", quote = F)

In [ ]:
################################################
### CC2931 - RECHECKING CQN NORMALIZED DATA ####
################################################
CC2931_cqn_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_noiseq_filtered_cqn_normalized_samescale.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE)
CC2931_noiseq_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'X')
cqn_res <- cqn(counts = CC2931_cqn_filtered, x = CC2931_noiseq_filtered$GC, lengths = CC2931_noiseq_filtered$total_exon_length)

factor <- colnames(counts)
columns <- factor
factor[grep('ANC', factor)] <- 'control'
factor[grep('L', factor)] <- 'treatment'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### CREATING PLOT ####
jpeg(file = "cqn/CC2931_gc_plot_after.jpeg")
cqnplot(cqn_res, n = 1, xlab = 'GC content', ylim = c(5,10))
dev.off()

jpeg(file = "cqn/CC2931_length_plot_after.jpeg")
cqnplot(cqn_res, n = 2, xlim = c(-2,4.5), xlab = 'length', ylim = c(5.5, 8.5))
dev.off()

################################################
### CC2344 - RECHECKING CQN NORMALIZED DATA ####
################################################
CC2344_cqn_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_noiseq_filtered_cqn_normalized_samescale.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE)
CC2344_noiseq_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'X')
cqn_res <- cqn(counts = CC2344_cqn_filtered, x = CC2344_noiseq_filtered$GC, lengths = CC2344_noiseq_filtered$total_exon_length)

factor <- colnames(counts)
columns <- factor
factor[grep('ANC', factor)] <- 'control'
factor[grep('L', factor)] <- 'treatment'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### CREATING PLOT ####
jpeg(file = "cqn/CC2344_gc_plot_after.jpeg")
cqnplot(cqn_res, n = 1, xlab = 'GC content', ylim = c(3,10))
dev.off()

jpeg(file = "cqn/CC2344_length_plot_after.jpeg")
cqnplot(cqn_res, n = 2, xlim = c(-2,4.5), xlab = 'length', ylim = c(4,8))
dev.off()

## DESEQ2 - DIFFERENTIAL EXPRESSION ANALYSIS

### CC2931

In [ ]:
#### USING DESEQ #####
dds <- DESeq(CC2931_dds1, parallel = TRUE)
res_L1 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L1'), alpha = 0.05)
res_L2 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L2'), alpha = 0.05)
res_L3 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L3'), alpha = 0.05)
res_L4 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L4'), alpha = 0.05)
res_L5 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L5'), alpha = 0.05)
res_L6 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L6'), alpha = 0.05)
res_L7 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L7'), alpha = 0.05)
res_L9 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L9'), alpha = 0.05)
res_L10 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L10'), alpha = 0.05)
res_L11 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L11'), alpha = 0.05)
res_L13 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L13'), alpha = 0.05)
res_L14 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L14'), alpha = 0.05)
res_L15 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L15'), alpha = 0.05)

#### SUMMARY OF PROFILE ####
resOrdered_L1 <- res_L1[order(res_L1$pvalue),]
resOrdered_L2 <- res_L2[order(res_L2$pvalue),]
resOrdered_L3 <- res_L3[order(res_L3$pvalue),]
resOrdered_L4 <- res_L4[order(res_L4$pvalue),]
resOrdered_L5 <- res_L5[order(res_L5$pvalue),]
resOrdered_L6 <- res_L6[order(res_L6$pvalue),]
resOrdered_L7 <- res_L7[order(res_L7$pvalue),]
resOrdered_L9 <- res_L9[order(res_L9$pvalue),]
resOrdered_L10 <- res_L10[order(res_L10$pvalue),]
resOrdered_L11 <- res_L11[order(res_L11$pvalue),]
resOrdered_L13 <- res_L13[order(res_L13$pvalue),]
resOrdered_L14 <- res_L14[order(res_L14$pvalue),]
resOrdered_L15 <- res_L15[order(res_L15$pvalue),]

# summary(resOrdered_L1)
# summary(resOrdered_L2)
# summary(resOrdered_L3)
# summary(resOrdered_L4)
# summary(resOrdered_L5)
# summary(resOrdered_L6)
# summary(resOrdered_L7)
# summary(resOrdered_L9)
# summary(resOrdered_L10)
# summary(resOrdered_L11)
# summary(resOrdered_L13)
# summary(resOrdered_L14)
# summary(resOrdered_L15)

# #### LFC SHRINKAGE ####
# resLFC_L1 <- lfcShrink(dds, coef="factor_L1_vs_ANC", type="apeglm")
# resLFC_L2 <- lfcShrink(dds, coef="factor_L2_vs_ANC", type="apeglm")
# resLFC_L3 <- lfcShrink(dds, coef="factor_L3_vs_ANC", type="apeglm")
# resLFC_L4 <- lfcShrink(dds, coef="factor_L4_vs_ANC", type="apeglm")
# resLFC_L5 <- lfcShrink(dds, coef="factor_L5_vs_ANC", type="apeglm")
# resLFC_L6 <- lfcShrink(dds, coef="factor_L6_vs_ANC", type="apeglm")
# resLFC_L7 <- lfcShrink(dds, coef="factor_L7_vs_ANC", type="apeglm")
# resLFC_L9 <- lfcShrink(dds, coef="factor_L9_vs_ANC", type="apeglm")
# resLFC_L10 <- lfcShrink(dds, coef="factor_L10_vs_ANC", type="apeglm")
# resLFC_L11 <- lfcShrink(dds, coef="factor_L11_vs_ANC", type="apeglm")
# resLFC_L13 <- lfcShrink(dds, coef="factor_L13_vs_ANC", type="apeglm")
# resLFC_L14 <- lfcShrink(dds, coef="factor_L14_vs_ANC", type="apeglm")
# resLFC_L15 <- lfcShrink(dds, coef="factor_L15_vs_ANC", type="apeglm")

#### FILES OF DIFFERNTIALLY EXPRESSED GENES ####
resSig_L1 = resOrdered_L1[which(resOrdered_L1$padj < 0.05),]
resSig_L2 = resOrdered_L2[which(resOrdered_L2$padj < 0.05),]
resSig_L3 = resOrdered_L3[which(resOrdered_L3$padj < 0.05),]
resSig_L4 = resOrdered_L4[which(resOrdered_L4$padj < 0.05),]
resSig_L5 = resOrdered_L5[which(resOrdered_L5$padj < 0.05),]
resSig_L6 = resOrdered_L6[which(resOrdered_L6$padj < 0.05),]
resSig_L7 = resOrdered_L7[which(resOrdered_L7$padj < 0.05),]
resSig_L9 = resOrdered_L9[which(resOrdered_L9$padj < 0.05),]
resSig_L10 = resOrdered_L10[which(resOrdered_L10$padj < 0.05),]
resSig_L11 = resOrdered_L11[which(resOrdered_L11$padj < 0.05),]
resSig_L13 = resOrdered_L13[which(resOrdered_L13$padj < 0.05),]
resSig_L14 = resOrdered_L14[which(resOrdered_L14$padj < 0.05),]
resSig_L15 = resOrdered_L15[which(resOrdered_L15$padj < 0.05),]

summary(resSig_L1)
summary(resSig_L2)
summary(resSig_L3)
summary(resSig_L4)
summary(resSig_L5)
summary(resSig_L6)
summary(resSig_L7)
summary(resSig_L9)
summary(resSig_L10)
summary(resSig_L11)
summary(resSig_L13)
summary(resSig_L14)
summary(resSig_L15)

# #### PLOTS ####
# plotMA(res_L1, ylim=c(-5,5))
# plotMA(resLFC_L1, ylim=c(-5,5))
# plotMA(res_L2, ylim=c(-5,5))
# plotMA(resLFC_L2, ylim=c(-5,5))
# plotMA(res_L3, ylim=c(-5,5))
# plotMA(resLFC_L3, ylim=c(-5,5))
# plotMA(res_L4, ylim=c(-5,5))
# plotMA(resLFC_L4, ylim=c(-5,5))
# plotMA(res_L5, ylim=c(-5,5))
# plotMA(resLFC_L5, ylim=c(-5,5))
# plotMA(res_L6, ylim=c(-5,5))
# plotMA(resLFC_L6, ylim=c(-5,5))
# plotMA(res_L7, ylim=c(-5,5))
# plotMA(resLFC_L7, ylim=c(-5,5))
# plotMA(res_L9, ylim=c(-5,5))
# plotMA(resLFC_L9, ylim=c(-5,5))
# plotMA(res_L10, ylim=c(-5,5))
# plotMA(resLFC_L10, ylim=c(-5,5))
# plotMA(res_L11, ylim=c(-5,5))
# plotMA(resLFC_L11, ylim=c(-5,5))
# plotMA(res_L13, ylim=c(-5,5))
# plotMA(resLFC_L13, ylim=c(-5,5))
# plotMA(res_L14, ylim=c(-5,5))
# plotMA(resLFC_L14, ylim=c(-5,5))
# plotMA(res_L15, ylim=c(-5,5))
# plotMA(resLFC_L15, ylim=c(-5,5))
# abline(h=c(-1,1), col="dodgerblue", lwd=2)

#### EXPORTING FILES OF DIFFERNTIALLY EXPRESSED GENES ####
write.csv(as.data.frame(resSig_L1), file="CC2931.resSig_L1.genes")
write.csv(as.data.frame(resSig_L2), file="CC2931.resSig_L2.genes")
write.csv(as.data.frame(resSig_L3), file="CC2931.resSig_L3.genes")
write.csv(as.data.frame(resSig_L4), file="CC2931.resSig_L4.genes")
write.csv(as.data.frame(resSig_L5), file="CC2931.resSig_L5.genes")
write.csv(as.data.frame(resSig_L6), file="CC2931.resSig_L6.genes")
write.csv(as.data.frame(resSig_L7), file="CC2931.resSig_L7.genes")
write.csv(as.data.frame(resSig_L9), file="CC2931.resSig_L9.genes")
write.csv(as.data.frame(resSig_L10), file="CC2931.resSig_L10.genes")
write.csv(as.data.frame(resSig_L11), file="CC2931.resSig_L11.genes")
write.csv(as.data.frame(resSig_L13), file="CC2931.resSig_L13.genes")
write.csv(as.data.frame(resSig_L14), file="CC2931.resSig_L14.genes")
write.csv(as.data.frame(resSig_L15), file="CC2931.resSig_L15.genes")

#### EXPORTING LOG2FOLD CHANGE RESULTS ####
write.table(as.data.frame(res_L1), 'genes_log2fold/CC2931-L1',  sep="\t", quote = F)
write.table(as.data.frame(res_L2), 'genes_log2fold/CC2931-L2',  sep="\t", quote = F)
write.table(as.data.frame(res_L3), 'genes_log2fold/CC2931-L3',  sep="\t", quote = F)
write.table(as.data.frame(res_L4), 'genes_log2fold/CC2931-L4',  sep="\t", quote = F)
write.table(as.data.frame(res_L5), 'genes_log2fold/CC2931-L5',  sep="\t", quote = F)
write.table(as.data.frame(res_L6), 'genes_log2fold/CC2931-L6',  sep="\t", quote = F)
write.table(as.data.frame(res_L7), 'genes_log2fold/CC2931-L7',  sep="\t", quote = F)
write.table(as.data.frame(res_L9), 'genes_log2fold/CC2931-L9',  sep="\t", quote = F)
write.table(as.data.frame(res_L10), 'genes_log2fold/CC2931-L10',  sep="\t", quote = F)
write.table(as.data.frame(res_L11), 'genes_log2fold/CC2931-L11',  sep="\t", quote = F)
write.table(as.data.frame(res_L13), 'genes_log2fold/CC2931-L13',  sep="\t", quote = F)
write.table(as.data.frame(res_L14), 'genes_log2fold/CC2931-L14',  sep="\t", quote = F)
write.table(as.data.frame(res_L15), 'genes_log2fold/CC2931-L15',  sep="\t", quote = F)

# #### PCA PLOT ####
# vsdata <- vst(dds, blind=FALSE)
# plotPCA(vsdata, intgroup="factor")

### CC2344

In [ ]:
#### USING DESEQ #####
dds <- DESeq(CC2344_dds1, parallel = TRUE)
res_L1 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L1'), alpha = 0.05)
res_L2 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L2'), alpha = 0.05)
res_L3 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L3'), alpha = 0.05)
res_L4 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L4'), alpha = 0.05)
res_L5 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L5'), alpha = 0.05)
res_L6 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L6'), alpha = 0.05)
res_L7 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L7'), alpha = 0.05)
res_L8 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L8'), alpha = 0.05)
res_L9 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L9'), alpha = 0.05)
res_L10 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L10'), alpha = 0.05)
res_L11 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L11'), alpha = 0.05)
res_L12 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L12'), alpha = 0.05)
res_L13 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L13'), alpha = 0.05)
res_L14 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L14'), alpha = 0.05)
res_L15 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L15'), alpha = 0.05)

#### SUMMARY OF PROFILE ####
resOrdered_L1 <- res_L1[order(res_L1$pvalue),]
resOrdered_L2 <- res_L2[order(res_L2$pvalue),]
resOrdered_L3 <- res_L3[order(res_L3$pvalue),]
resOrdered_L4 <- res_L4[order(res_L4$pvalue),]
resOrdered_L5 <- res_L5[order(res_L5$pvalue),]
resOrdered_L6 <- res_L6[order(res_L6$pvalue),]
resOrdered_L7 <- res_L7[order(res_L7$pvalue),]
resOrdered_L8 <- res_L8[order(res_L8$pvalue),]
resOrdered_L9 <- res_L9[order(res_L9$pvalue),]
resOrdered_L10 <- res_L10[order(res_L10$pvalue),]
resOrdered_L11 <- res_L11[order(res_L11$pvalue),]
resOrdered_L12 <- res_L12[order(res_L12$pvalue),]
resOrdered_L13 <- res_L13[order(res_L13$pvalue),]
resOrdered_L14 <- res_L14[order(res_L14$pvalue),]
resOrdered_L15 <- res_L15[order(res_L15$pvalue),]

# summary(resOrdered_L1)
# summary(resOrdered_L2)
# summary(resOrdered_L3)
# summary(resOrdered_L4)
# summary(resOrdered_L5)
# summary(resOrdered_L6)
# summary(resOrdered_L7)
# summary(resOrdered_L8)
# summary(resOrdered_L9)
# summary(resOrdered_L10)
# summary(resOrdered_L11)
# summary(resOrdered_L12)
# summary(resOrdered_L13)
# summary(resOrdered_L14)
# summary(resOrdered_L15)

# #### LFC SHRINKAGE ####
# resLFC_L1 <- lfcShrink(dds, coef="factor_L1_vs_ANC", type="apeglm")
# resLFC_L2 <- lfcShrink(dds, coef="factor_L2_vs_ANC", type="apeglm")
# resLFC_L3 <- lfcShrink(dds, coef="factor_L3_vs_ANC", type="apeglm")
# resLFC_L4 <- lfcShrink(dds, coef="factor_L4_vs_ANC", type="apeglm")
# resLFC_L5 <- lfcShrink(dds, coef="factor_L5_vs_ANC", type="apeglm")
# resLFC_L6 <- lfcShrink(dds, coef="factor_L6_vs_ANC", type="apeglm")
# resLFC_L7 <- lfcShrink(dds, coef="factor_L7_vs_ANC", type="apeglm")
# resLFC_L8 <- lfcShrink(dds, coef="factor_L8_vs_ANC", type="apeglm")
# resLFC_L9 <- lfcShrink(dds, coef="factor_L9_vs_ANC", type="apeglm")
# resLFC_L10 <- lfcShrink(dds, coef="factor_L10_vs_ANC", type="apeglm")
# resLFC_L11 <- lfcShrink(dds, coef="factor_L11_vs_ANC", type="apeglm")
# resLFC_L8 <- lfcShrink(dds, coef="factor_L12_vs_ANC", type="apeglm")
# resLFC_L13 <- lfcShrink(dds, coef="factor_L13_vs_ANC", type="apeglm")
# resLFC_L14 <- lfcShrink(dds, coef="factor_L14_vs_ANC", type="apeglm")
# resLFC_L15 <- lfcShrink(dds, coef="factor_L15_vs_ANC", type="apeglm")

#### FILES OF DIFFERNTIALLY EXPRESSED GENES ####
resSig_L1 = resOrdered_L1[which(resOrdered_L1$padj < 0.05),]
resSig_L2 = resOrdered_L2[which(resOrdered_L2$padj < 0.05),]
resSig_L3 = resOrdered_L3[which(resOrdered_L3$padj < 0.05),]
resSig_L4 = resOrdered_L4[which(resOrdered_L4$padj < 0.05),]
resSig_L5 = resOrdered_L5[which(resOrdered_L5$padj < 0.05),]
resSig_L6 = resOrdered_L6[which(resOrdered_L6$padj < 0.05),]
resSig_L7 = resOrdered_L7[which(resOrdered_L7$padj < 0.05),]
resSig_L8 = resOrdered_L8[which(resOrdered_L8$padj < 0.05),]
resSig_L9 = resOrdered_L9[which(resOrdered_L9$padj < 0.05),]
resSig_L10 = resOrdered_L10[which(resOrdered_L10$padj < 0.05),]
resSig_L11 = resOrdered_L11[which(resOrdered_L11$padj < 0.05),]
resSig_L12 = resOrdered_L12[which(resOrdered_L12$padj < 0.05),]
resSig_L13 = resOrdered_L13[which(resOrdered_L13$padj < 0.05),]
resSig_L14 = resOrdered_L14[which(resOrdered_L14$padj < 0.05),]
resSig_L15 = resOrdered_L15[which(resOrdered_L15$padj < 0.05),]

summary(resSig_L1)
summary(resSig_L2)
summary(resSig_L3)
summary(resSig_L4)
summary(resSig_L5)
summary(resSig_L6)
summary(resSig_L7)
summary(resSig_L8)
summary(resSig_L9)
summary(resSig_L10)
summary(resSig_L11)
summary(resSig_L12)
summary(resSig_L13)
summary(resSig_L14)
summary(resSig_L15)

# #### PLOTS ####
# plotMA(res_L1, ylim=c(-5,5))
# plotMA(resLFC_L1, ylim=c(-5,5))
# plotMA(res_L2, ylim=c(-5,5))
# plotMA(resLFC_L2, ylim=c(-5,5))
# plotMA(res_L3, ylim=c(-5,5))
# plotMA(resLFC_L3, ylim=c(-5,5))
# plotMA(res_L4, ylim=c(-5,5))
# plotMA(resLFC_L4, ylim=c(-5,5))
# plotMA(res_L5, ylim=c(-5,5))
# plotMA(resLFC_L5, ylim=c(-5,5))
# plotMA(res_L6, ylim=c(-5,5))
# plotMA(resLFC_L6, ylim=c(-5,5))
# plotMA(res_L7, ylim=c(-5,5))
# plotMA(resLFC_L7, ylim=c(-5,5))
# plotMA(res_L8, ylim=c(-5,5))
# plotMA(resLFC_L8, ylim=c(-5,5))
# plotMA(res_L9, ylim=c(-5,5))
# plotMA(resLFC_L9, ylim=c(-5,5))
# plotMA(res_L10, ylim=c(-5,5))
# plotMA(resLFC_L10, ylim=c(-5,5))
# plotMA(res_L11, ylim=c(-5,5))
# plotMA(resLFC_L11, ylim=c(-5,5))
# plotMA(res_L12, ylim=c(-5,5))
# plotMA(resLFC_L12, ylim=c(-5,5))
# plotMA(res_L13, ylim=c(-5,5))
# plotMA(resLFC_L13, ylim=c(-5,5))
# plotMA(res_L14, ylim=c(-5,5))
# plotMA(resLFC_L14, ylim=c(-5,5))
# plotMA(res_L15, ylim=c(-5,5))
# plotMA(resLFC_L15, ylim=c(-5,5))
# abline(h=c(-1,1), col="dodgerblue", lwd=2)

#### EXPORTING FILES OF DIFFERNTIALLY EXPRESSED GENES ####
write.csv(as.data.frame(resSig_L1), file="CC2344.resSig_L1.genes")
write.csv(as.data.frame(resSig_L2), file="CC2344.resSig_L2.genes")
write.csv(as.data.frame(resSig_L3), file="CC2344.resSig_L3.genes")
write.csv(as.data.frame(resSig_L4), file="CC2344.resSig_L4.genes")
write.csv(as.data.frame(resSig_L5), file="CC2344.resSig_L5.genes")
write.csv(as.data.frame(resSig_L6), file="CC2344.resSig_L6.genes")
write.csv(as.data.frame(resSig_L7), file="CC2344.resSig_L7.genes")
write.csv(as.data.frame(resSig_L8), file="CC2344.resSig_L8.genes")
write.csv(as.data.frame(resSig_L9), file="CC2344.resSig_L9.genes")
write.csv(as.data.frame(resSig_L10), file="CC2344.resSig_L10.genes")
write.csv(as.data.frame(resSig_L11), file="CC2344.resSig_L11.genes")
write.csv(as.data.frame(resSig_L12), file="CC2344.resSig_L12.genes")
write.csv(as.data.frame(resSig_L13), file="CC2344.resSig_L13.genes")
write.csv(as.data.frame(resSig_L14), file="CC2344.resSig_L14.genes")
write.csv(as.data.frame(resSig_L15), file="CC2344.resSig_L15.genes")

#### EXPORTING LOG2FOLD CHANGE RESULTS ####
write.table(as.data.frame(res_L1), 'genes_log2fold/CC2344-L1',  sep="\t", quote = F)
write.table(as.data.frame(res_L2), 'genes_log2fold/CC2344-L2',  sep="\t", quote = F)
write.table(as.data.frame(res_L3), 'genes_log2fold/CC2344-L3',  sep="\t", quote = F)
write.table(as.data.frame(res_L4), 'genes_log2fold/CC2344-L4',  sep="\t", quote = F)
write.table(as.data.frame(res_L5), 'genes_log2fold/CC2344-L5',  sep="\t", quote = F)
write.table(as.data.frame(res_L6), 'genes_log2fold/CC2344-L6',  sep="\t", quote = F)
write.table(as.data.frame(res_L7), 'genes_log2fold/CC2344-L7',  sep="\t", quote = F)
write.table(as.data.frame(res_L8), 'genes_log2fold/CC2344-L8',  sep="\t", quote = F)
write.table(as.data.frame(res_L9), 'genes_log2fold/CC2344-L9',  sep="\t", quote = F)
write.table(as.data.frame(res_L10), 'genes_log2fold/CC2344-L10',  sep="\t", quote = F)
write.table(as.data.frame(res_L11), 'genes_log2fold/CC2344-L11',  sep="\t", quote = F)
write.table(as.data.frame(res_L12), 'genes_log2fold/CC2344-L12',  sep="\t", quote = F)
write.table(as.data.frame(res_L13), 'genes_log2fold/CC2344-L13',  sep="\t", quote = F)
write.table(as.data.frame(res_L14), 'genes_log2fold/CC2344-L14',  sep="\t", quote = F)
write.table(as.data.frame(res_L15), 'genes_log2fold/CC2344-L15',  sep="\t", quote = F)

# # #PCA PLOT
# # vsdata <- vst(dds, blind=FALSE)
# # plotPCA(vsdata, intgroup="condition")